In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import unicodedata

def edit_comment(comment):
    replacements = {"&#252": "ü", "&#220": "Ü", "&#231": "ç", "&#252": "Ç", "&#246": "ö", "&#214": "Ö"}
    for key, value in replacements.items():
        comment = comment.replace(key, value)
    return comment

def remove_emojis(comment):
    cleaned_text = ''
    for char in comment:
        if not is_emoji(char):
            cleaned_text += char
    return cleaned_text

def is_emoji(character):
    if unicodedata.category(character) == 'So':
        return True
    if unicodedata.category(character) == 'Sk' and not character.isprintable():
        return True
    return False

def parenthesisControl(comment):
    return comment.translate({ord("("): None, ord(")"): None})

def write_comments_to_file(comments, file_count):
    with open(f"comments_{file_count}.txt", "a", encoding="utf-8") as file:
        for comment in comments:
            rating_element = comment.find('span', class_='rating')
            if rating_element:
                rating = rating_element['class'][1].replace('r', '')
            else:
                rating = '0'
            
            comment_text = comment.find('p').text.strip()
            comment_text = edit_comment(comment_text)
            comment_text = parenthesisControl(comment_text)
            comment_text = remove_emojis(comment_text)
            file.write(str(int(rating) / 100) + "\t'" + comment_text + "'\n")

# Chrome WebDriver'ı başlat
driver = webdriver.Chrome()
driver.implicitly_wait(3)  # Sayfa yüklenirken 10 saniye bekleyin

# Ürünlerin URL'lerini içeren dosyayı aç
with open("products_links_continue.txt", "r") as f:
    lines = f.read().split('\n')

# Dosya sayacı
file_count = 1

for line in lines:
    driver.get(line)

    # Ürün sayacı
    product_count = 0

    while True:
        # Sayfa kaynağını alın
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comments = soup.find_all('li', class_='comment')

        # Yorumları dosyaya yaz
        write_comments_to_file(comments, file_count)

        if product_count >= 270:
            file_count += 1
            product_count = 0
            break

        # Sonraki sayfaya geçmek için kontrol et
        try:
            next_button = driver.find_element(By.CLASS_NAME, 'next')
            driver.execute_script("arguments[0].click();", next_button)
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "comment")))
        except:
            product_count += len(comments)
            break

# WebDriver'ı kapat
driver.quit()


In [ ]:
#test code



In [ ]:
# bos urunleri ve 0 olan yorumlari alma filitrele

In [ ]:
belirli urun sayisindan sonra gelen sorumlari  aska dosyaya yazmak icin gerekli kod blogu

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import unicodedata

def edit_comment(comment):
    return comment.replace("&#252","ü").replace("&#220","Ü").replace("&#231","ç").replace("&#252","Ç").replace("&#246","ö").replace("&#214","Ö")

def remove_emojis(comment):
    cleaned_text = ''
    for char in comment:
        if not is_emoji(char):
            cleaned_text += char
    return cleaned_text

def is_emoji(character):
    if unicodedata.category(character) == 'So':
        return True
    if unicodedata.category(character) == 'Sk' and not character.isprintable():
        return True
    return False
    
def parenthesisControl(comment):
    return comment.replace("(","").replace(")","")

file_count = 1
product_count = 0  # Başlangıçta ürün sayısını 0 olarak ayarlayın.

with open("productsLink.txt", "r") as f:
    lines = f.read().split('\n')

for line in lines:
    driver = webdriver.Chrome()
    driver.get(line)

    while True:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comments = soup.find_all('li', class_='comment')

        # Her 30 ürün için yeni bir dosya oluştur
        if product_count >= 30:
            file_count += 1
            product_count = 0
            driver.quit()  # Her 30 üründen sonra sürücüyü kapatın
            break

        with open(f"comments_{file_count}.txt", "a", encoding="utf-8") as file:
            for comment in comments:
                rating = comment.find('span', class_='rating')['class'][1].replace('r', '')
                comment_text = comment.find('p').text.strip()
                comment_text = edit_comment(comment_text)
                comment_text = parenthesisControl(comment_text)
                comment_text = remove_emojis(comment_text)
                file.write(str(int(rating) / 100) + "\t'" + comment_text + "'\n")

        try:
            next_button = driver.find_element(By.CLASS_NAME, 'next')
            driver.execute_script("arguments[0].click();", next_button)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "comment")))
        except:
            product_count += 1  # Bir sonraki dosyaya geçmeden önce ürün sayısını artırın
            break
